# Import CSV files, combine them and extract Numpy arrays

In [81]:
import numpy as np
import pandas as pd
import json
import os
import glob
import re
from ast import literal_eval

### Import files into pd.DataFrame

In [82]:
dname = os.getcwd() + '/data_v2/'
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2/'

In [83]:
# list the file names
fnames = glob.glob(dname + '*.txt')
fnames

['c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\0tl442hvKm05.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\1S2NaIDr2Hb7.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\3dWkc3IGEK2s.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\3opDXh8RdT6k.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\4NSH4GA1Oqqx.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\4R36AQWVXsxM.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\7aw0AQVPy0Mn.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\7JFi4csQY00N.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\7sp6iH0i4S3G.txt',
 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2\\8uloC73yqgyF.txt',
 'c:\\Users\\Jirko\\Desktop\\H

In [84]:
# regular expressions to make the file suitable to be read into a dataframe
def replace_several(diction, text):
    # create regex from dict
    regex = re.compile("(%s)" % "|".join(map(re.escape, diction.keys())))

    # look-up value in dictionary
    return regex.sub(lambda mo: diction[mo.string[mo.start():mo.end()]], text)  

# dictionary for replacement
expres_dict = {"sdata": '\"sdata\"', "edata": '\"edata\"', "parameters": '\"parameters\"'}

# run a loop over file names
for fname in fnames:
    # make a new file to place the data
    newfile = open("data_v2_quotes/"+ os.path.basename(os.path.normpath(fname)) , 'w')
    # print("data_v2_quotes/"+ os.path.basename(os.path.normpath(fname)))
    for i, line in enumerate(open(fname)):
        rep_line = replace_several(expres_dict, line)
        # replace the content of the file line by line
        newfile.write("%s" % rep_line)
        # newfile.write('\n')

In [85]:
#  function from https://github.com/aranas
def data2dict(file_path):
    """ Load in data saved from javascript online study
    Deals with missing '{}' and outputs data in the form of dicts.
    ​
    Parameters:
    path_to_file (str) : full path to file that contains data
​   
    Returns:
    out (list) : list of dicts from file
    ​
    """
    with open(file_path, "r") as file:
        data_read = file.read().split('\n')
    out = [json.loads('{' + line + '}') for line in data_read]
    return out

# test = data2dict("data_v2_quotes/3dWkc3IGEK2s.txt")
# print(len(test))
# print(test[])


In [86]:
dname_new = os.getcwd() + '/data_v2_quotes/'
fnames_new = glob.glob(dname_new + '*.txt')
print(fnames_new)

['c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\0tl442hvKm05.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\1S2NaIDr2Hb7.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\3dWkc3IGEK2s.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\3opDXh8RdT6k.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\4NSH4GA1Oqqx.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\4R36AQWVXsxM.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\7aw0AQVPy0Mn.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\7JFi4csQY00N.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data_v2_quotes\\7sp6iH0i4S3G.txt', 'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\data_loading/data

In [87]:
DF = pd.DataFrame()
for i, fname in enumerate(fnames):

    with open(fname) as f:
        # read the data and transform to a dictionary
        data = f.read() 
        js = json.loads(data)
        print(len(js["sdata"]))
        # enter the dict into a dataframe
        df = pd.DataFrame.from_dict(js["sdata"], orient='index')
        df = df.transpose() # small hack to load data into dataframe
        # add a participant ID
        df.insert(loc=0, column="id", value=i)
    
    # Append to the big dataFrame
    DF = DF.append(df, ignore_index=True)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [88]:
def create_df(fnames):
    """function to create the dataframe from the files"""
    DF = pd.DataFrame()
    for i, fname in enumerate(fnames):
        list_of_dicts = data2dict(fname)
        # get the relevant sdata
        data_dict = list_of_dicts[0]["sdata"]
        
        # remove entries of type integer
        for key in list(data_dict):
            if isinstance(data_dict[key], int):
                del data_dict[key]

        # creating the dict
        df = pd.DataFrame.from_dict(data_dict, orient='index')
        df = df.transpose() # small hack to load data into dataframe
        # add a participant ID
        df.insert(loc=0, column="id", value=i)
    
        # Append to the big dataFrame
        DF = DF.append(df, ignore_index=True)
    return DF 

DF = create_df(fnames_new)


C:\Users\Jirko\AppData\Local\Temp\ipykernel_15220\2468291895.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF = DF.append(df, ignore_index=True)
C:\Users\Jirko\AppData\Local\Temp\ipykernel_15220\2468291895.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF = DF.append(df, ignore_index=True)
C:\Users\Jirko\AppData\Local\Temp\ipykernel_15220\2468291895.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF = DF.append(df, ignore_index=True)
C:\Users\Jirko\AppData\Local\Temp\ipykernel_15220\2468291895.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF = DF.append(df, ignore_index=True)
C:\Users\Jirko\AppData\Local\Temp\ip

### Explore DataFrame

In [89]:
pd.set_option('display.max_columns', None)
# DF.drop(columns=["trial_bonus", "block_bonus", "resp_indices", "resp_indices_values", "resp_timestamp", "resp_reactiontime"], inplace=True)
DF.shape
DF.head(6)

,id,expt_index,expt_trial,expt_block,trial_att,vbxi_category,exemplar,trialProperties,trialPropShuff,relevProps,relevPropsShuff,trial_bonus,block_bonus,resp_timestamp,resp_reactiontime,property_order,resp_correct_array,resp_correct_array_unshuf,prop_responses,groundTruthCorrect,groundTruthCorrect_unshuf,resp_indices,resp_indices_values
0,0,0,0,0,1,7,10,"[1, 0, 1, 0, 0, 0, 1]","[[0, 1, 0, 0, 1, 0, 1], [0, 0, 0, 1, 1, 1, 0]]",None,[],1,23,"[1656927914973, 1656927926827]","[9.699, 21.552]","[[1, 0, 5, 4, 2, 3, 6], [5, 3, 4, 2, 0, 6, 1]]","[[False, True, False, None, None, None, None],...","[[True, False, None, None, None, False, None],...","[[1, 1, 1, None, None, None, None], [None, Non...","[[0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1]]","[[1, 0, 0, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 2, 1], [4, 3, 5]]","[[1, 1, 1], [1, 1, 1]]"
1,0,1,1,0,0,7,0,"[1, 0, 1, 0, 0, 0, 1]","[[1, 1, 1, 0, 0, 0, 0]]",None,[],3,32,[1656927934679],[5.846],"[[0, 6, 2, 3, 1, 5, 4]]","[[True, True, True, None, None, None, None]]","[[True, None, True, None, None, None, True]]","[[1, 1, 1, None, None, None, None]]","[[1, 1, 1, 1, 1, 1, 1]]","[[1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 2]]","[[1, 1, 1]]"
2,0,2,2,0,1,3,1,"[1, 1, 0, 0, 1, 0, 0]","[[1, 0, 0, 0, 1, 0, 1], [1, 1, 1, 0, 0, 0, 0]]",None,[],1,31,"[1656927943664, 1656927953941]","[6.981, 17.258]","[[4, 2, 5, 3, 1, 6, 0], [0, 4, 1, 2, 5, 3, 6]]","[[None, None, False, False, True, None, None],...","[[None, True, None, False, None, False, None],...","[[None, None, 1, 1, 1, None, None], [1, 1, 1, ...","[[0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[2, 4, 3], [1, 2, 0]]","[[1, 1, 1], [1, 1, 1]]"
3,0,3,3,0,1,1,1,"[1, 1, 0, 1, 0, 0, 0]","[[1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0]]",None,[],1,41,"[1656927965718, 1656927975115]","[9.773, 19.169]","[[0, 1, 3, 4, 6, 5, 2], [6, 0, 4, 1, 3, 5, 2]]","[[None, None, True, False, None, False, None],...","[[None, None, None, True, False, False, None],...","[[None, None, 1, 1, None, 1, None], [None, 1, ...","[[0, 0, 1, 0, 1, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[0, 0, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[5, 2, 3], [1, 3, 4]]","[[1, 1, 1], [1, 1, 1]]"
4,0,4,4,0,1,4,10,"[1, 0, 1, 0, 0, 1, 0]","[[0, 0, 0, 1, 0, 1, 1], [1, 1, 0, 0, 0, 0, 1]]",None,[],0,38,"[1656927990198, 1656928000956]","[13.08, 23.838]","[[3, 4, 6, 5, 1, 2, 0], [0, 2, 6, 4, 3, 1, 5]]","[[False, False, None, None, False, None, None]...","[[None, False, None, False, False, None, None]...","[[1, 1, None, None, 1, None, None], [1, 1, Non...","[[0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[0, 4, 1], [6, 1, 0]]","[[1, 1, 1], [1, 1, 1]]"
5,0,5,5,0,1,1,9,"[1, 1, 0, 1, 0, 0, 0]","[[0, 0, 1, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0]]",None,[],2,38,"[1656928010793, 1656928021026]","[7.832, 18.065]","[[6, 4, 0, 1, 2, 3, 5], [0, 3, 5, 6, 2, 1, 4]]","[[None, None, True, None, False, True, None], ...","[[True, None, False, True, None, None, None], ...","[[None, None, 1, None, 1, 1, None], [1, 1, Non...","[[1, 1, 1, 0, 0, 1, 1], [1, 1, 1, 1, 1, 1, 1]]","[[1, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]","[[2, 4, 5], [1, 0, 5]]","[[1, 1, 1], [1, 1, 1]]"


### Filter responses

In [90]:
# drop redundant columns
DF.drop(columns=["relevProps", "relevPropsShuff"], inplace=True)
DF.head(15)

,id,expt_index,expt_trial,expt_block,trial_att,vbxi_category,exemplar,trialProperties,trialPropShuff,trial_bonus,block_bonus,resp_timestamp,resp_reactiontime,property_order,resp_correct_array,resp_correct_array_unshuf,prop_responses,groundTruthCorrect,groundTruthCorrect_unshuf,resp_indices,resp_indices_values
0,0,0,0,0,1,7,10,"[1, 0, 1, 0, 0, 0, 1]","[[0, 1, 0, 0, 1, 0, 1], [0, 0, 0, 1, 1, 1, 0]]",1,23,"[1656927914973, 1656927926827]","[9.699, 21.552]","[[1, 0, 5, 4, 2, 3, 6], [5, 3, 4, 2, 0, 6, 1]]","[[False, True, False, None, None, None, None],...","[[True, False, None, None, None, False, None],...","[[1, 1, 1, None, None, None, None], [None, Non...","[[0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1]]","[[1, 0, 0, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 2, 1], [4, 3, 5]]","[[1, 1, 1], [1, 1, 1]]"
1,0,1,1,0,0,7,0,"[1, 0, 1, 0, 0, 0, 1]","[[1, 1, 1, 0, 0, 0, 0]]",3,32,[1656927934679],[5.846],"[[0, 6, 2, 3, 1, 5, 4]]","[[True, True, True, None, None, None, None]]","[[True, None, True, None, None, None, True]]","[[1, 1, 1, None, None, None, None]]","[[1, 1, 1, 1, 1, 1, 1]]","[[1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 2]]","[[1, 1, 1]]"
2,0,2,2,0,1,3,1,"[1, 1, 0, 0, 1, 0, 0]","[[1, 0, 0, 0, 1, 0, 1], [1, 1, 1, 0, 0, 0, 0]]",1,31,"[1656927943664, 1656927953941]","[6.981, 17.258]","[[4, 2, 5, 3, 1, 6, 0], [0, 4, 1, 2, 5, 3, 6]]","[[None, None, False, False, True, None, None],...","[[None, True, None, False, None, False, None],...","[[None, None, 1, 1, 1, None, None], [1, 1, 1, ...","[[0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[2, 4, 3], [1, 2, 0]]","[[1, 1, 1], [1, 1, 1]]"
3,0,3,3,0,1,1,1,"[1, 1, 0, 1, 0, 0, 0]","[[1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0]]",1,41,"[1656927965718, 1656927975115]","[9.773, 19.169]","[[0, 1, 3, 4, 6, 5, 2], [6, 0, 4, 1, 3, 5, 2]]","[[None, None, True, False, None, False, None],...","[[None, None, None, True, False, False, None],...","[[None, None, 1, 1, None, 1, None], [None, 1, ...","[[0, 0, 1, 0, 1, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[0, 0, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[5, 2, 3], [1, 3, 4]]","[[1, 1, 1], [1, 1, 1]]"
4,0,4,4,0,1,4,10,"[1, 0, 1, 0, 0, 1, 0]","[[0, 0, 0, 1, 0, 1, 1], [1, 1, 0, 0, 0, 0, 1]]",0,38,"[1656927990198, 1656928000956]","[13.08, 23.838]","[[3, 4, 6, 5, 1, 2, 0], [0, 2, 6, 4, 3, 1, 5]]","[[False, False, None, None, False, None, None]...","[[None, False, None, False, False, None, None]...","[[1, 1, None, None, 1, None, None], [1, 1, Non...","[[0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[0, 4, 1], [6, 1, 0]]","[[1, 1, 1], [1, 1, 1]]"
5,0,5,5,0,1,1,9,"[1, 1, 0, 1, 0, 0, 0]","[[0, 0, 1, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0]]",2,38,"[1656928010793, 1656928021026]","[7.832, 18.065]","[[6, 4, 0, 1, 2, 3, 5], [0, 3, 5, 6, 2, 1, 4]]","[[None, None, True, None, False, True, None], ...","[[True, None, False, True, None, None, None], ...","[[None, None, 1, None, 1, 1, None], [1, 1, Non...","[[1, 1, 1, 0, 0, 1, 1], [1, 1, 1, 1, 1, 1, 1]]","[[1, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]","[[2, 4, 5], [1, 0, 5]]","[[1, 1, 1], [1, 1, 1]]"
6,0,6,6,0,1,2,8,"[1, 1, 0, 0, 1, 0, 0]","[[0, 0, 0, 1, 0, 1, 1], [1, 1, 0, 0, 0, 1, 0]]",1,36,"[1656928030375, 1656928041180]","[7.344, 18.149]","[[6, 2, 3, 1, 5, 0, 4], [1, 0, 2, 6, 5, 4, 3]]","[[None, None, False, True, False, None, None],...","[[None, True, None, False, None, False, None],...","[[None, None, 1, 1, 1, None, None], [1, 1, Non...","[[1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[3, 4, 2], [0, 1, 5]]","[[1, 1, 1], [1, 1, 1]]"
7,0,7,7,0,3,0,2,"[1, 1, 0, 1, 0, 0, 0]","[[1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 1, 0, 0, 1],...",2,36,"[1656928050809, 1656928061198, 1656928077118, ...","[7.6240000000000006, 18.013, 33.933, 43.394]","[[1, 5, 2, 3, 0, 4, 6], [1, 4, 2, 0, 6, 5, 3],...","[[None, None, False, True, True, None, None], ...","[[True, None, False, True, None, None, None], ...","[[None, None,

### Save full DataFrame to CSV

In [91]:
DF.to_csv(dname_new + 'full_data.csv')

### Retreive parameters

In [92]:
sujs = DF['id'].unique()
n_sujs = sujs.size
n_sujs

49

In [93]:
n_trials_per_suj = DF.loc[DF['id'] == sujs[0]].shape[0]
n_trials_per_suj

128

In [94]:
n_blocs = DF['expt_block'].unique().size
n_blocs

8

In [95]:
n_trials_per_block = int(n_trials_per_suj/n_blocs)
n_trials_per_block

16

### Put data in numpy arrays

In [99]:
ACCURACY = np.full((n_sujs, n_blocs, n_trials_per_block), np.nan)
acc_first_prop = np.full((n_sujs, n_blocs, n_trials_per_block), np.nan)
acc_second_prop = np.full((n_sujs, n_blocs, n_trials_per_block), np.nan)
acc_third_prop = np.full((n_sujs, n_blocs, n_trials_per_block), np.nan)
# RT = np.full((n_sujs, n_blocs, n_trial), np.nan)
# BLOC_TYPE = np.full((n_sujs, n_blocs), np.nan, dtype=np.object)

# Loop across participant and blocs
for i, suj in enumerate(sujs):
    for block in range(n_blocs):

        # Select lines in the DataFrame
        _DF = DF.loc[(DF['id'] == suj) & (DF['expt_block'] == block)]

        _DF["Correct"] = _DF["resp_correct_array_unshuf"].apply(lambda x: np.asarray(x)[0])
        _DF["Correct"] = _DF["Correct"].apply(lambda x: np.select([x == True],[1]))

        # Accuracy
        ACCURACY[i, block, :_DF.shape[0]] = _DF["Correct"].apply(lambda x: np.sum(x)/3)
        # accuracy on first property
        acc_first_prop[i, block, :_DF.shape[0]] = _DF["Correct"].apply(lambda x: np.sum(x[0]))
        
        # accuracy on second property
        acc_second_prop[i, block, :_DF.shape[0]] = _DF["Correct"].apply(lambda x: np.sum(x[1:3]))
        # accuracy on second property
        acc_third_prop[i, block, :_DF.shape[0]] = _DF["Correct"].apply(lambda x: np.sum(x[3:]))
        # Reaction times
        # RT[i, j, :_DF.shape[0]] = _DF['data : "rt"']
        
        # Bloc type
        # BLOC_TYPE[i, j] = _DF['bloc_type'].unique()[0]

C:\Users\Jirko\AppData\Local\Temp\ipykernel_15220\972292794.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _DF["Correct"] = _DF["resp_correct_array_unshuf"].apply(lambda x: np.asarray(x)[0])
C:\Users\Jirko\AppData\Local\Temp\ipykernel_15220\972292794.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _DF["Correct"] = _DF["Correct"].apply(lambda x: np.select([x == True],[1]))
C:\Users\Jirko\AppData\Local\Temp\ipykernel_15220\972292794.py:15: SettingWithCopyWarning: 
A value is trying to be set on a c

### Save data as numpy arrays

In [100]:
np.save(dname_new + 'ACCURACY_New.npy', ACCURACY)
np.save(dname_new + 'Acc_first_prop_New.npy', acc_first_prop)
np.save(dname_new + 'acc_second_prop_New.npy', acc_second_prop)
np.save(dname_new + 'acc_third_prop_New.npy', acc_third_prop)
# np.save(dname + 'SUJS.npy', sujs)